In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import os
import pytorch_lightning
import pytorchvideo.data
import torch.utils.data
import pandas as pd
import csv
import json
from typing import List
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from PIL import Image
from pytorchvideo.data.encoded_video import EncodedVideo
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import Video
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
def get_duration(filename):
    video = cv2.VideoCapture(filename)

    duration = video.get(cv2.CAP_PROP_POS_MSEC)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    
    video = EncodedVideo.from_path(filename)

    return int(video.duration), int(frame_count)

def cast_pred_to_timestamps(pred: np.array) -> list[str]:
    timestamps = []
    is_open = False
    for i in range(0, pred.size):
        if (pred[i] == 0):
            if (i != 0 and pred[i-1] == 0):
                is_open = False
        elif (is_open):
            continue
        else:
            is_open = True
            timestamps.append(f"{i // 3600}:{i % 3600 // 60}:{i % 60}")    
    return timestamps

In [4]:
train_dir = 'data/train'
val_dir = 'data/val'
target_dir = 'data/target'

train_files = os.listdir(train_dir)[1:]
val_files = os.listdir(val_dir)[:-1]

In [6]:
DATA_DIR = val_dir

In [175]:
#sol zeros
accur = 0
for file in val_files:
    inp = open(os.path.join(target_dir, file + '.txt'), 'r')
    target = np.array(list(map(int, inp.readline().split())))
    pred = np.zeros(target.shape[0])
    accur += np.square(pred - target).mean() / len(val_files)

1 - accur, accur

(0.9830050827725246, 0.01699491722747537)

In [381]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

model = YOLO("yolov8n.yaml").load("yolov8n.pt")
model.export(format="onnx")

Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.2.27 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Core Processor (Broadwell, no TSX))
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.yaml' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (0.0 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 1.1s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (2.6s)
Results saved to /home/jupyter/work/resources
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=None  
Visualize:       https://netron.app


'yolov8n.onnx'

In [177]:
def predict(filename):
    dur, frame_count = get_duration(filename)
    results = model(filename, verbose=False, stream=True, conf=0.6)
    fps = frame_count/dur
    predict = np.zeros(dur)
    i = 0
    print(frame_count)
    for result in tqdm(results):
        boxes = result.boxes  # Boxes object for bounding box outputs    
        if 0. in boxes.cls:
            predict[int(i // fps)] = 1.
        i += 1
    return predict

8992


8992it [01:39, 90.13it/s]


4305


4305it [00:47, 90.41it/s]


424


...

139


139it [00:01, 91.13it/s]


204


204it [00:02, 93.03it/s]


(0.9107901594407779, 0.08920984055922214, 0.4482758620689655, 29)

In [171]:
accur = 0
accur_1 = 0
count_1 = 0
for file in train_files:
    inp = open(os.path.join(target_dir, file + '.txt'), 'r')
    target = np.array(list(map(int, inp.readline().split())))
    pred = predict(os.path.join(train_dir, file))
    lenn = min(pred.shape[0], target.shape[0])
    
    target = target[:lenn]
    pred = pred[:lenn]
    
    accur += np.abs(pred - target).mean() / len(train_files)
    print(np.abs(pred - target).mean())
    
    count_1 += (target == 1).sum()
    accur_1 += pred[target == 1].sum()

1 - accur, accur, accur_1 / count_1, count_1

304


304it [00:03, 89.21it/s]


0.16666666666666666
253


253it [00:02, 87.82it/s]


0.12
209


...

0.0
215


215it [00:02, 90.57it/s]


0.42857142857142855
219


93it [00:01, 88.45it/s]


KeyboardInterrupt: 

In [367]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

model = YOLO("yolov8n.yaml").load("yolov8n.pt")

Transferred 355/355 items from pretrained weights


In [277]:
def addData(filename):
    target_file = open(os.path.join(target_dir, filename + '.txt'), 'r')
    target = np.array(list(map(int, target_file.readline().split())))
    
    dur, frame_count = get_duration(os.path.join(train_dir, filename))
    results = model(os.path.join(train_dir, filename), verbose=False, stream=True, conf=0.4)
    fps = frame_count/dur
    i = 0
    for result in tqdm(results):
        if i // fps == target.shape[0]:
            break
        
        boxes = result.boxes
        if (len(boxes) > 0):
            X.append(np.zeros(81))
            X[-1][0] = target[int(i // fps)]
            for cl in boxes.cls:
                X[-1][int(cl.item()) + 1] = 1
        i += 1

In [278]:
X = []

for file in train_files:
    addData(file)

304it [00:03, 91.44it/s]
253it [00:02, 89.24it/s]
209it [00:02, 93.02it/s]
290it [00:03, 88.36it/s]
304it [00:03, 85.64it/s]
205it [00:02, 91.80it/s]
305it [00:03, 84.83it/s]
305it [00:03, 88.04it/s]
308it [00:03, 88.38it/s]
201it [00:02, 91.22it/s]
209it [00:02, 91.28it/s]
190it [00:02, 90.75it/s]
289it [00:03, 92.18it/s]
224it [00:02, 85.33it/s]
204it [00:02, 91.19it/s]
207it [00:02, 94.14it/s]
285it [00:03, 87.99it/s]
923it [00:10, 89.34it/s]
231it [00:02, 87.47it/s]
215it [00:02, 89.76it/s]
219it [00:02, 91.43it/s]
204it [00:02, 90.33it/s]
8992it [01:41, 88.92it/s]
4135it [00:46, 89.82it/s]
1409it [00:16, 86.80it/s]
8992it [01:41, 88.71it/s]
8992it [01:42, 87.91it/s]
3275it [00:35, 91.26it/s]
8992it [01:39, 90.35it/s]
3148it [00:35, 89.29it/s]


In [279]:
df = pd.DataFrame(X)
df.to_csv("objects_train.csv")

In [280]:
def addData(filename):
    target_file = open(os.path.join(target_dir, filename + '.txt'), 'r')
    target = np.array(list(map(int, target_file.readline().split())))
    
    dur, frame_count = get_duration(os.path.join(val_dir, filename))
    results = model(os.path.join(val_dir, filename), verbose=False, stream=True, conf=0.4)
    fps = frame_count/dur
    i = 0
    for result in tqdm(results):
        if i // fps == target.shape[0]:
            break
        
        boxes = result.boxes
        if (len(boxes) > 0):
            X.append(np.zeros(81))
            X[-1][0] = target[int(i // fps)]
            for cl in boxes.cls:
                X[-1][int(cl.item()) + 1] = 1
        i += 1

In [380]:
onnx_model = YOLO("yolov8n.onnx")

FileNotFoundError: 'yolov8n.onnx' does not exist

In [281]:
X = []

for file in val_files:
    addData(file)
    
df = pd.DataFrame(X)
df.to_csv("objects_val.csv")

8992it [01:39, 90.61it/s]
4305it [00:47, 90.03it/s]
424it [00:04, 85.61it/s]
274it [00:02, 93.00it/s]
236it [00:02, 93.11it/s]
139it [00:01, 91.60it/s]
204it [00:02, 91.80it/s]


In [374]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

model = YOLO("yolov8n.yaml").load("yolov8n.pt")

Transferred 355/355 items from pretrained weights


In [375]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

df_train = pd.read_csv("objects.csv")
df_val = pd.read_csv("objects_val.csv")

X_train = df_train.values[:, 2:].astype(int)
y_train = df_train.values[:, 1].astype(int)

X_val = df_val.values[:, 2:].astype(int)
y_val = df_val.values[:, 1].astype(int)

classifier = RandomForestClassifier(class_weight={1: 9.2, 0: 1})

In [376]:
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_val)
print(1 - np.abs(y_pred - y_val).mean())

0.42358078602620086


In [377]:
y_pred.sum(), y_val.sum(),  y_pred[y_val == 1].sum(), y_pred[y_val == 1].sum() / y_val.sum(), (1 - y_pred[y_val == 0]).sum() / (1 - y_val).sum()

(2695, 89, 72, 0.8089887640449438, 0.41594299710532173)

In [378]:
def predict(filename):
    dur, frame_count = get_duration(filename)
    results = model(filename, verbose=False, stream=True, conf=0.6)
    fps = frame_count/dur
    predict = np.zeros(dur)
    i = 0
    print(frame_count)
    for result in tqdm(results):
        boxes = result.boxes  # Boxes object for bounding box outputs    
        if (len(boxes) > 0):
            arr = np.zeros(80).astype(int)
            for cl in boxes.cls:
                arr[int(cl.item())] = 1
            predict[int(i // fps)] = classifier.predict([arr]).item()
        i += 1
    return predict

In [379]:
accur = 0
accur_1 = 0
count_1 = 0
for file in val_files:
    inp = open(os.path.join(target_dir, file + '.txt'), 'r')
    target = np.array(list(map(int, inp.readline().split())))
    pred = predict(os.path.join(val_dir, file))
    lenn = min(pred.shape[0], target.shape[0])
    
    target = target[:lenn]
    pred = pred[:lenn]
    
    accur += np.abs(pred - target).mean() / len(val_files)
    
    count_1 += (target == 1).sum()
    accur_1 += pred[target == 1].sum()

1 - accur, accur, accur_1 / count_1, count_1

8992


8992it [01:47, 83.66it/s]


4305


4305it [00:50, 84.51it/s]


424


...

139


139it [00:01, 89.22it/s]


204


204it [00:02, 93.58it/s]


(0.9064135726832511, 0.09358642731674895, 0.4482758620689655, 29)